In [1]:
import geopandas as gpd
import os
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial import Polynomial
import pandas as pd
import warnings
from shapely.ops import unary_union, cascaded_union
from shapely.validation import make_valid
import folium
import contextily as ctx
import networkx as nx
from tqdm import tqdm
from shapely.geometry import Point, MultiPolygon, Polygon
from shapely.ops import unary_union
import geopandas.tools

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)



# GPS - DHS

In [ ]:


# Directory containing the GeoJSON files
geojson_dir = '/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/maps-master/LGD Villages/'

# List all GeoJSON files in the directory
geojson_files = [os.path.join(geojson_dir, file) for file in os.listdir(geojson_dir) if file.endswith('.geojson')]

# Load all GeoJSON files into a list of GeoDataFrames
gdfs = [gpd.read_file(file) for file in geojson_files]

# Concatenate all GeoDataFrames into a single GeoDataFrame
merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))


In [ ]:
merged_gdf2 = merged_gdf.copy()
# Drop the specified columns
columns_to_drop = [
    'OBJECTID', 'vilnam_soi', 'vilcode11', 'vilname11', 'vil_lgd', 
    'block_name', 'block_lgd', 'Remark', 
    'ac_no', 'test', 
]
merged_gdf2 = merged_gdf2.drop(columns=columns_to_drop)

# Function to clean invalid geometries
def clean_geometry(geom):
    if not geom.is_valid:
        return make_valid(geom)
    return geom

# Clean invalid geometries
merged_gdf2['geometry'] = merged_gdf2['geometry'].apply(clean_geometry)

# Optional: further clean geometries by buffering and unbuffering (can help with slight geometry errors)
merged_gdf2['geometry'] = merged_gdf2.buffer(0)

# Define the aggregation functions
agg_functions = {
    'shape_Length': 'sum',
    'shape_Area': 'sum'
    # Add more columns with 'first' if you want to retain them without aggregation
}

# Add all other columns with 'first' to retain their first occurrence
for col in merged_gdf2.columns:
    if col not in agg_functions and col != 'geometry':
        agg_functions[col] = 'first'

# Dissolve (merge) villages into subdistricts based on the "subdt_lgd" attribute, summing 'shape_Length' and 'shape_Area'
dissolved_gdf = merged_gdf2.dissolve(by=['stname','dtname','sdtname','gp_code', 'gp_name',], aggfunc=agg_functions)


# Optionally, save the dissolved GeoDataFrame to a new GeoJSON file
# output_file = os.path.join(geojson_dir, 'merged_subdistricts.geojson')
# dissolved_gdf.to_file(output_file, driver='GeoJSON')

# print("Merged and dissolved GeoDataFrame saved to:", output_file)
dissolved_gdf = dissolved_gdf.drop(columns={'stname','dtname','sdtname','gp_code', 'gp_name',})
dissolved_gdf = dissolved_gdf.reset_index()

ts = dissolved_gdf[dissolved_gdf['stname']=='TELANGANA']


dissolved_gdf

In [ ]:
ax = dissolved_gdf.plot(figsize=(10, 10), edgecolor='k', alpha=0.5)

# Set plot title and labels
plt.title('Dissolved GeoDataFrame Plot')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Optional: Add a legend if you have categorical data or specific attributes to show
# plt.legend(title='Your Legend Title')

# Show the plot
plt.show()

In [ ]:
nfhs5 = gpd.read_file('/Users/sid/Desktop/IAGE7AFL/IAGE7AFL.shp')
india = gpd.read_file('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/maps-master/Country/india.geojson')

# Ensure both GeoDataFrames have the same CRS
if india.crs != nfhs5.crs:
    nfhs5 = nfhs5.to_crs(india.crs)

# Perform spatial join to keep only points within India
nfhs5_within_india = gpd.sjoin(nfhs5, india, op='within')

# Dropping the 'index_right' column created by the spatial join
nfhs5_within_india = nfhs5_within_india.drop(columns='index_right')

# Save or further process the filtered GeoDataFrame
# nfhs5_within_india.to_file('path_to_filtered_nfhs5_shapefile')

print(f"Number of points within India: {len(nfhs5_within_india)}")
print(f"Number of points outside India: {len(nfhs5) - len(nfhs5_within_india)}")

buffer_distance = 0.04  # 4 km 
nfhs5_within_india['geometry'] = nfhs5_within_india.geometry.buffer(buffer_distance)


In [ ]:
# Perform spatial join to identify intersections
print("Performing spatial join...")
joined_gdf2 = gpd.sjoin(dissolved_gdf, nfhs5_within_india, how='inner', op='intersects')

# Calculate the area of each village
print("Calculating village areas...")
dissolved_gdf['village_area'] = dissolved_gdf.area

# Calculate the area of intersection for each village with nfhs5_within_india polygons
print("Calculating intersection areas...")
intersections = []
for i in tqdm(dissolved_gdf.index, desc="Overlay operation"):
    village = dissolved_gdf.loc[[i]]
    intersection = gpd.overlay(village, nfhs5_within_india, how='intersection')
    intersection['village_area'] = village['village_area'].values[0]
    intersection['intersection_area'] = intersection.area
    intersections.append(intersection)

# Concatenate the list of GeoDataFrames into a single GeoDataFrame
intersections_gdf = gpd.GeoDataFrame(pd.concat(intersections, ignore_index=True))

# Calculate the proportion of each village's area that lies within each intersecting nfhs5_within_india polygon
print("Calculating proportions...")
intersections_gdf['proportion'] = intersections_gdf['intersection_area'] / intersections_gdf['village_area']

In [ ]:
# Select the polygon with the highest proportion for each village
print("Selecting highest proportion for each village...")
intersections_gdf = intersections_gdf.sort_values(by='proportion', ascending=False).drop_duplicates(subset=['stname', 'dtname', 'sdtname', 'gp_name', 'gp_code'], keep='first')
intersections_gdf.columns
Final_gp_df = intersections_gdf.drop(columns={'shape_Length','shape_Area', 
       'ADM1FIPS', 'ADM1FIPSNA', 'ADM1SALBNA', 'ADM1SALBCO', 'ADM1DHS',
       'URBAN_RURA','LONGNUM', 'ALT_GPS', 'ALT_DEM', 'DATUM', 'geometry',})

In [ ]:
# Final_gp_df.to_csv('/Users/sid/Desktop/DHS/dhs_gp_prop.csv',index=False)

In [ ]:


ax = nfhs5_within_india.plot(figsize=(10, 10), edgecolor='k', alpha=0.5)

# Set plot title and labels
plt.title('Dissolved GeoDataFrame Plot')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Optional: Add a legend if you have categorical data or specific attributes to show
# plt.legend(title='Your Legend Title')

# Show the plot
plt.show()

In [ ]:
# Ensure the CRS is in WGS84 for Folium

# Create a folium map centered on India
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5)


# Add the nfhs5_within_india GeoDataFrame to the map
folium.GeoJson(
    intersections_gdf,
    style_function=lambda feature: {
        'fillColor': 'none',  # No fill
        'color': 'blue',  # Border color
        'weight': 1.5,  # Border width
    }
).add_to(m)

# Add the dissolved_gdf GeoDataFrame to the map
folium.GeoJson(
    ts,
    style_function=lambda feature: {
        'fillColor': 'none',  # No fill
        'color': 'red',  # Border color
        'weight': 1.5,  # Border width
    }
).add_to(m)

# Add tile layer for background
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Watercolor').add_to(m)
folium.TileLayer('cartodb positron').add_to(m)
folium.TileLayer('cartodb dark_matter').add_to(m)
folium.TileLayer('openstreetmap').add_to(m)
folium.LayerControl().add_to(m)
m


In [ ]:
# Ensure the CRS is in WGS84 for Folium
nfhs5_within_india = nfhs5_within_india.to_crs(epsg=4326)

# Create a folium map centered on India
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5)


# Add the nfhs5_within_india GeoDataFrame to the map
folium.GeoJson(
    nfhs5_within_india,
    style_function=lambda feature: {
        'fillColor': 'none',  # No fill
        'color': 'blue',  # Border color
        'weight': 1.5,  # Border width
    }
).add_to(m)

# Add the dissolved_gdf GeoDataFrame to the map
folium.GeoJson(
    ts,
    style_function=lambda feature: {
        'fillColor': 'none',  # No fill
        'color': 'red',  # Border color
        'weight': 1.5,  # Border width
    }
).add_to(m)

# Add tile layer for background
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Watercolor').add_to(m)
folium.TileLayer('cartodb positron').add_to(m)
folium.TileLayer('cartodb dark_matter').add_to(m)
folium.TileLayer('openstreetmap').add_to(m)
folium.LayerControl().add_to(m)
m


In [ ]:
joined_gdf = gpd.sjoin(nfhs5, dissolved_gdf, how='left', op='within')
columns_to_drop = [
    'DHSID','DHSCC','DHSYEAR','CCFIPS',
 'ADM1FIPS',
 'ADM1FIPSNA',
 'ADM1SALBNA',
 'ADM1SALBCO',
 'ADM1DHS',
 'geometry',
#  'index_right',
]

joined_gdf = joined_gdf.drop(columns=columns_to_drop)

joined_gdf = joined_gdf.dropna(subset=['stname'])
polygons_with_points = joined_gdf[['index_right', 'gp_name','gp_code']].drop_duplicates()

# joined_gdf.to_csv('/Users/sid/Desktop/nfhs_gp.csv',index=False)

joined_gdf

# Stata

## DF clean

In [ ]:
import pandas as pd

# Specify the path to your .dta file
file_path = '/Users/sid/Desktop/DHS/BR/BR.DTA'

# Step 1: Read the Stata file to get the column names
initial_df = pd.read_stata(file_path, convert_categoricals=False)

# Identify the columns to drop (for example, 'SDIST')
columns_to_drop = ['sdist']

# Step 2: Read the Stata file again, dropping the problematic column(s) and converting categoricals
df = pd.read_stata(file_path, convert_categoricals=True, columns=[col for col in initial_df.columns if col not in columns_to_drop])

# Display the first few rows of the DataFrame
df

In [ ]:
unique_values = df['v602'].unique()
print(unique_values)

In [ ]:
df_cleaned = df.dropna(axis=1, how='all')
df_cleaned

In [ ]:
# df_cleaned2 = df_cleaned[df_cleaned['b2']>=2017]
df_cleaned['period'] = np.where(df_cleaned['b2'] < 2017, 0, 
                                    np.where(df_cleaned['b2'] >= 2017, 1, np.nan))

In [ ]:
# df_cleaned2.to_csv('/Users/sid/Desktop/DHS/BR/BR_python_filter.csv',index=False)
df_cleaned.to_csv('/Users/sid/Desktop/DHS/BR/BR_python_all.csv',index=False)

## DF read

In [5]:
# df_cleaned2 = pd.read_csv('/Users/sid/Desktop/DHS/BR/BR_python_filter.csv')
df_cleaned = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/DHS/BR/BR_python_all.csv')

In [ ]:


df_cleaned_filter = df_cleaned[['v001','v002',
    's361', 'v106', 
    'v133', 'v149', 'v701','s365a','s365b' ,'s365c', 's365d', 's365e', 's365g', 's365j', 
    's365o', 's365n', 's365p', 's365q', 's365r', 's566a','s566b','s566c', 'v602', 'v201', 
    'v613', 's370a', 's370b','s370c', 's370d','s370e', 's370f','s370g', 's370h', 's370i', 
    's370j','s370k', 's370l','v010', 'v009', 'v212', 's929', 'v130', 'v131','v739', 'v741',  
    'v714','v717','v719', 'v731', 'v741', 'm2h','m2g', 'm2i','m2j','m2k', 'period','b2'
]]

df_cleaned_filter2 = df_cleaned_filter.rename(columns={
                        'v001':'Cluster number','v002':'Household number','v739':'Person who decides how to spend respondents earnings', 
                        'v741':'Type of earnings from respondents work', 'v714':'Respondent currently working', 
                        'v717':'Type of Employment',
                        'v719':'Respondent works for family, others, self', 
                        'v731':'Respondent worked in last 12 months', 'm2h':'Prenatal: community/village health worker',
                        'm2g':'Prenatal: DAI/traditional birth attendant' ,'m2i':'Prenatal: anganwadi/ICDS worker', 'm2j':'Prenatal: ASHA', 'm2k':'Prenatal: other',
                        's361':'Met with an anganwadi worker, ASHA or other community health worker in last 3 mo', 'v106':'Highest educational level', 
                        'v133':'Education in single years', 'v149':'Educational attainment', 'v701':'Husband education level',
                        's365a': 'Services: Family Planning', 's365b': 'Services: Immunization', 's365c': 'Services: Antenatal Care', 's365d': 'Services: Delivery Care',
                        's365e': 'Services: Birth Preparedness', 's365g': 'Services: Postnatal Care', 's365j': 'Services: Treatment For Sick Child',
                        's365o': 'Services: Early Childhood Care', 's365n': 'Services: Growth Monitoring Of Child', 's365p': 'Services: Pre-School Education',
                        's365q': 'Services: Nutrition/Health Education', 's365r': 'Services: Family Life Education',
                        's566a':'Benefits:Supplementary food','s566b':'Benefits:Health check-ups','s566c':'Benefits:Health and nutrition education', 
                        'v602':'Fertility preference', 'v201':'Total children ever born', 'v613':'Ideal number of children', 
                        's370a': 'Service went for: Family Planning', 's370b': 'Service went for: Immunization', 's370c': 'Service went for: Antenatal Care',
                        's370d': 'Service went for: Delivery Care', 's370e': 'Service went for: Postnatal Care', 's370f': 'Service went for: Disease Prevention',
                        's370g': 'Service went for: Medical Treatment For Self', 's370h': 'Service went for: Treatment For Child', 
                        's370i': 'Service went for: Treatment For Other Person',  's370j': 'Service went for: Growth Monitoring Of Child',
                        's370k': 'Service went for: Health Check-Up', 's370l': 'Service went for: Medical Termination Of Pregnancy',
                        'v010':'Respondents year of birth', 'v009':'Respondents month of birth', 'v212':'Age of respondent at 1st birth', 's929':'Independent money decision', 
                        'v130':'Religion', 'v131':'Ethnicity','b2':'Child birth year'
                                    })
df_cleaned_filter2


In [ ]:
df_cleaned_filter2 = df_cleaned_filter2.astype('object')
df_cleaned_filter2['Cluster number'] = df_cleaned_filter2['Cluster number'].astype('int')
df_cleaned_filter2['Household number'] = df_cleaned_filter2['Household number'].astype('int')
df_cleaned_filter2['Education in single years'] = df_cleaned_filter2['Education in single years'].astype('int')
df_cleaned_filter2['Ideal number of children'] = pd.to_numeric(df_cleaned_filter2['Ideal number of children'], errors='coerce')
df_cleaned_filter2['Total children ever born'] = df_cleaned_filter2['Total children ever born'].astype('Int64')
df_cleaned_filter2['Respondents year of birth'] = df_cleaned_filter2['Respondents year of birth'].astype('Int64')
df_cleaned_filter2['Respondents month of birth'] = df_cleaned_filter2['Respondents month of birth'].astype('Int64')
df_cleaned_filter2['Age of respondent at 1st birth'] = df_cleaned_filter2['Age of respondent at 1st birth'].astype('Int64')
df_cleaned_filter2['period'] = df_cleaned_filter2['period'].astype('object')
print(df_cleaned_filter2.dtypes)

In [ ]:
# df_cleaned_filter2.to_csv('/Users/sid/Desktop/DHS/BR/BR_python_filter.csv',index=False)

# Test Join

In [3]:
sub = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/DHS/dhs_subd.csv')
gp = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/DHS/dhs_gp.csv')
prop = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/DHS/dhs_gp_prop.csv')
dhs = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/DHS/BR/BR_python_filter.csv')
meg_sorted = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/ma2020/2020_Villages_sorted.csv')
lum12_sorted = pd.read_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/ma2020/2011_Villages_sorted.csv')
# df_cleaned = pd.read_csv('/Users/sid/Desktop/DHS/BR/BR_python_all.csv')

lum12_sorted['sc_pop_share']=lum12_sorted['sc_pop']/lum12_sorted['population']
lum12_sorted['st_pop_share']=lum12_sorted['st_pop']/lum12_sorted['population']
meg_sorted['sc_pop'] = None
meg_sorted['st_pop'] = None
meg_sorted['sc_pop_share'] = None
meg_sorted['st_pop_share'] = None

meg_sorted['year']=2020
lum12_sorted['year']=2011


# Handle duplicate column names by keeping the first occurrence
meg_sorted = meg_sorted.loc[:, ~meg_sorted.columns.duplicated()]

# Check for unique column names
if not meg_sorted.columns.is_unique or not lum12_sorted.columns.is_unique:
    raise ValueError("One of the DataFrames has non-unique column names.")

# Find the common columns
common_columns = meg_sorted.columns.intersection(lum12_sorted.columns)

# Subset both DataFrames to include only these common columns
meg20_common = meg_sorted[common_columns]
lum11_common = lum12_sorted[common_columns]

gp = gp[['DHSCLUST','stname','dtname','sdtname','gp_code','gp_name','dist_lgd','state_lgd','subdt_lgd']]



In [3]:
meg20 = meg20_common[['state','district','subdistrict','gp','gpcode', 'vil_code',
                      'Village','state_lgd','dist_lgd','subdt_lgd','village_area_sqkm',
                      'SubDistrict Area', 'avg_ntl','population', 'male_pop', 'female_pop', 
                      'sc_pop', 'st_pop', 'year', 'p_school','m_school', 
                      'h_school', 'ssc_school', 'arts_and_science_degree_college','iti', 
                      'phc', 'dispensary', 'mcwc', 'veterinary_hospital', 'aanganwadi',
                      'cooperative_bank', 'post_office', 'bus', 'railway', 'all_weather_road','mandis'
       ]]

lum11 = lum11_common[['state','district','subdistrict','gp','gpcode', 'vil_code',
                      'Village','state_lgd','dist_lgd','subdt_lgd','village_area_sqkm',
                      'SubDistrict Area', 'avg_ntl','population', 'male_pop', 'female_pop', 
                      'sc_pop', 'st_pop', 'year', 'p_school','m_school', 
                      'h_school', 'ssc_school', 'arts_and_science_degree_college','iti', 
                      'phc', 'dispensary', 'mcwc', 'veterinary_hospital', 'aanganwadi',
                      'cooperative_bank', 'post_office', 'bus', 'railway', 'all_weather_road','mandis'
       ]]

meg20 = meg20.drop(columns='vil_code')
lum11 = lum11.drop(columns='vil_code')


# Assuming meg20 is your DataFrame
columns_to_exclude = ['state_lgd', 'dist_lgd', 'subdt_lgd', 'SubDistrict Area','year']
group_columns = ['state', 'district', 'subdistrict', 'gp', 'gpcode','Village']

# Create a dictionary for aggregation
agg_dict = {col: 'sum' for col in meg20.columns if col not in columns_to_exclude + group_columns}
# For the excluded columns, use 'first' to keep the first occurrence
for col in columns_to_exclude:
    agg_dict[col] = 'first'

# Perform the groupby and aggregation
meg = meg20.groupby(['state', 'district', 'subdistrict', 'gp', 'gpcode']).agg(agg_dict).reset_index()
lum = lum11.groupby(['state', 'district', 'subdistrict', 'gp', 'gpcode']).agg(agg_dict).reset_index()

# meg['year']=2020
# meg = meg.assign(year=2020)
# meg

In [50]:
dhs_gp_join = pd.merge(dhs,gp,left_on='Cluster number',right_on='DHSCLUST',how='left')
dhs_gp_join = dhs_gp_join.dropna(subset=['DHSCLUST','gp_name'])
dhs_gp_join = dhs_gp_join.rename(columns={
                                        'stname':'state',
                                        'dtname':'district',
                                        'sdtname':'subdistrict',
                                        'gp_name':'gp',
                                        'gp_code':'gpcode',
                                        })

dhs_post = dhs_gp_join[dhs_gp_join['period'] == 1 ]
dhs_pre = dhs_gp_join[dhs_gp_join['period'] == 0 ]


dhs_ma_post = pd.merge(dhs_post,meg,on=['state','state_lgd', 'district', 'dist_lgd',
                                            'subdistrict', 'subdt_lgd','gp', 'gpcode'],how='left')
dhs_ma_pre = pd.merge(dhs_pre,lum,on=['state','state_lgd', 'district', 'dist_lgd',
                                            'subdistrict', 'subdt_lgd','gp', 'gpcode'],how='left')
dhs_ma_join = pd.concat([dhs_ma_post,dhs_ma_pre])
dhs_ma_join = dhs_ma_join.dropna(subset=['year'])
# dhs_ma_join


In [51]:
state_to_zone = {
    'ANDHRA PRADESH': 'A',
    'ASSAM': 'B',
    'BIHAR': 'B',
    'CHHATTISGARH': 'B',
    'GOA': 'A',
    'GUJARAT': 'A',
    'HARYANA': 'B',
    'JHARKHAND': 'B',
    'KARNATAKA': 'A',
    'KERALA': 'A',
    'MADHYA PRADESH': 'B',
    'MAHARASHTRA': 'A',
    'ODISHA': 'B',
    'PUNJAB': 'B',
    'RAJASTHAN': 'B',
    'TAMIL NADU': 'A',
    'TELANGANA': 'A',
    'TRIPURA':'B',
    'UTTAR PRADESH': 'B',
    'UTTARAKHAND': 'B',
    'WEST BENGAL': 'B'
}


dhs_ma_join['Region'] = dhs_ma_join['state'].map(state_to_zone)
dhs_ma_join

,Cluster number,Household number,"Met with an anganwadi worker, ASHA or other community health worker in last 3 mo",Highest educational level,Education in single years,Educational attainment,Husband education level,Services: Family Planning,Services: Immunization,Services: Antenatal Care,Services: Delivery Care,Services: Birth Preparedness,Services: Postnatal Care,Services: Treatment For Sick Child,Services: Early Childhood Care,Services: Growth Monitoring Of Child,Services: Pre-School Education,Services: Nutrition/Health Education,Services: Family Life Education,Benefits:Supplementary food,Benefits:Health check-ups,Benefits:Health and nutrition education,Fertility preference,Total children ever born,Ideal number of children,Service went for: Family Planning,Service went for: Immunization,Service went for: Antenatal Care,Service went for: Delivery Care,Service went for: Postnatal Care,Service went for: Disease Prevention,Service went for: Medical Treatment For Self,Service went for: Treatment For Child,Service went for: Treatment For Other Person,Service went for: Growth Monitoring Of Child,Service went for: Health Check-Up,Service went for: Medical Termination Of Pregnancy,Respondents year of birth,Respondents month of birth,Age of respondent at 1st birth,Independent money decision,Religion,Ethnicity,Person who decides how to spend respondents earnings,Type of earnings from respondents work,Respondent currently working,"Respondent works for family, others, self",Respondent worked in last 12 months,Type of earnings from respondents work.1,Prenatal: community/village health worker,Prenatal: DAI/traditional birth attendant,Prenatal: anganwadi/ICDS worker,Prenatal: ASHA,Prenatal: other,period,Child birth year,DHSCLUST,state,district,subdistrict,gpcode,gp,dist_lgd,state_lgd,subdt_lgd,village_area_sqkm,avg_ntl,population,male_pop,female_pop,sc_pop,st_pop,p_school,m_school,h_school,ssc_school,arts_and_science_degree_college,iti,phc,dispensary,mcwc,veterinary_hospital,aanganwadi,cooperative_bank,post_office,bus,railway,all_weather_road,mandis,SubDistrict Area,year,Region
0,3613,83,yes,higher,15,higher,NaN,no,no,no,no,no,no,no,no,no,no,no,no,yes,no,no,no more,2,2.0,no,no,no,no,no,no,no,yes,no,no,no,no,1991,12,21,NaN,sikh,caste,NaN,NaN,NaN,NaN,NaN,NaN,no,no,no,no,no,1.0,2018,3613.0,PUNJAB,Kapurthala,Kapurthala,15814.0,MANDER DONA,35.0,3.0,206.0,3.448564,21.010001,235.0,120.0,115.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,894.119726,2020.0,B
1,3630,80,yes,secondary,12,incomplete secondary,secondary,no,no,no,no,no,no,no,no,no,no,no,no,yes,yes,yes,undecided,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1993,1,26,no,sikh,caste,NaN,NaN,no,NaN,no,NaN,no,no,no,no,no,1.0,2019,3630.0,PUNJAB,Jalandhar,Jalandhar - II,14927.0,ATHOLA,34.0,3.0,213.0,6.922049,59.587777,2400.0,1202.0,1198.0,0,0,3.0,3.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,741.609605,2020.0,B
2,3630,63,yes,secondary,12,incomplete secondary,NaN,no,yes,no,no,no,no,no,no,no,no,no,no,yes,no,no,declared infecund,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990,7,28,NaN,sikh,caste,NaN,NaN,NaN,NaN,NaN,NaN,no,no,no,no,no,1.0,2019,3630.0,PUNJAB,Jalandhar,Jalandhar - II,14927.0,ATHOLA,34.0,3.0,213.0,6.922049,59.587777,2400.0,1202.0,1198.0,0,0,3.0,3.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,741.609605,2020.0,B
3,3630,75,no,secondary,12,incomplete secondary,secondary,no,no,no,no,no,no,no,no,no,no,no,no,yes,yes,yes,no more,2,2.0,no,no,no,no,no,no,no,no,no,no,no,no,1984,6,29,yes,sikh,caste,respondent alone,cash only,yes,for family member,currently working,cash only,no,no,no,no,no,1.0,2019,3630.0,PUNJAB,Jalandhar,Jalandhar - II,14927.0,ATHOLA,34.0,3.0,213.0,6.922049,59.587777,2400.0,1202.0,1198.0,0,0,3.0,3.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,741.609605,2020.0,B
4,3606,27,no,secondary,10,incomplete secondary,NaN,no,no,no,no,no,no,no,no,no,no,no,no,yes,yes,yes,no more,2,2.0,no,no,no,no,

In [52]:

dhs_ma2 = dhs_ma_join.drop_duplicates(subset={'year','state','district','subdistrict','gp','gpcode'})
dhs_ma2 = dhs_ma2[['year','state','district','subdistrict','gp','gpcode','mcwc']]
pivot_df = dhs_ma2.pivot(index=['state','district','subdistrict','gp','gpcode'], columns='year', values=['mcwc']).reset_index()

# Rename columns for clarity
pivot_df.columns = ['state','district','subdistrict','gp','gpcode', 'iti_2011', 'iti_2020']

# Create the 'treated' column based on the comparison of mcwc values
pivot_df['treated'] = np.where(pivot_df['iti_2011'] >= pivot_df['iti_2020'], 0, 1)
# pivot_df['mcwc_dens_2011']=pivot_df['iti_2011']/pivot_df['num_2011']
# pivot_df['mcwc_dens_2020']=pivot_df['iti_2020']/pivot_df['num_2020']
# pivot_df['intensity']=pivot_df['iti_2020']-pivot_df['iti_2011']
# pivot_df['density']=pivot_df['mcwc_dens_2020']-pivot_df['mcwc_dens_2011']

# Adjust intensity based on the conditions
# pivot_df['intensity'] = pivot_df.apply(
#     lambda row: 0 if row['intensity'] < 0 or row['iti_2011'] == row['iti_2020'] else row['intensity'], 
#     axis=1
# )
# pivot_df['density'] = pivot_df.apply(
#     lambda row: 0 if row['density'] < 0  else row['density'], 
#     axis=1
# )
# Drop duplicates to avoid issues in the merge step
# dissorted_sync = dissorted_sync.drop_duplicates(subset=['vil_code', 'year'])

# Merge the treated column back to the original DataFrame
dhs_ma2 = pd.merge(dhs_ma2, pivot_df[['state','district','subdistrict','gp','gpcode', 'treated','iti_2011', 'iti_2020'
                                                    # 'intensity','mcwc_dens_2020','mcwc_dens_2011','density'
                                                    ]]
                                , on=['state','district','subdistrict','gp','gpcode'], how='left')
dhs_ma2['treated'] = np.where(dhs_ma2['iti_2020'].isna() | dhs_ma2['iti_2011'].isna(), 0, dhs_ma2['treated'])
dhs_ma2 = dhs_ma2[['year','state','district','subdistrict','gp','gpcode','mcwc','treated']]

print(len(dhs_ma2))

dhs_ma_join = pd.merge(dhs_ma_join, dhs_ma2, on= ['state','district','subdistrict','gp','gpcode','mcwc','year'], how='left')
dhs_ma_join['unique_id'] = dhs_ma_join['state'] + '_' + dhs_ma_join['district'] + '_' + dhs_ma_join['subdistrict']+ '_' + dhs_ma_join['gp']
dhs_ma_join

# dhs_ma2

31243


,Cluster number,Household number,"Met with an anganwadi worker, ASHA or other community health worker in last 3 mo",Highest educational level,Education in single years,Educational attainment,Husband education level,Services: Family Planning,Services: Immunization,Services: Antenatal Care,Services: Delivery Care,Services: Birth Preparedness,Services: Postnatal Care,Services: Treatment For Sick Child,Services: Early Childhood Care,Services: Growth Monitoring Of Child,Services: Pre-School Education,Services: Nutrition/Health Education,Services: Family Life Education,Benefits:Supplementary food,Benefits:Health check-ups,Benefits:Health and nutrition education,Fertility preference,Total children ever born,Ideal number of children,Service went for: Family Planning,Service went for: Immunization,Service went for: Antenatal Care,Service went for: Delivery Care,Service went for: Postnatal Care,Service went for: Disease Prevention,Service went for: Medical Treatment For Self,Service went for: Treatment For Child,Service went for: Treatment For Other Person,Service went for: Growth Monitoring Of Child,Service went for: Health Check-Up,Service went for: Medical Termination Of Pregnancy,Respondents year of birth,Respondents month of birth,Age of respondent at 1st birth,Independent money decision,Religion,Ethnicity,Person who decides how to spend respondents earnings,Type of earnings from respondents work,Respondent currently working,"Respondent works for family, others, self",Respondent worked in last 12 months,Type of earnings from respondents work.1,Prenatal: community/village health worker,Prenatal: DAI/traditional birth attendant,Prenatal: anganwadi/ICDS worker,Prenatal: ASHA,Prenatal: other,period,Child birth year,DHSCLUST,state,district,subdistrict,gpcode,gp,dist_lgd,state_lgd,subdt_lgd,village_area_sqkm,avg_ntl,population,male_pop,female_pop,sc_pop,st_pop,p_school,m_school,h_school,ssc_school,arts_and_science_degree_college,iti,phc,dispensary,mcwc,veterinary_hospital,aanganwadi,cooperative_bank,post_office,bus,railway,all_weather_road,mandis,SubDistrict Area,year,Region,treated,unique_id
0,3613,83,yes,higher,15,higher,NaN,no,no,no,no,no,no,no,no,no,no,no,no,yes,no,no,no more,2,2.0,no,no,no,no,no,no,no,yes,no,no,no,no,1991,12,21,NaN,sikh,caste,NaN,NaN,NaN,NaN,NaN,NaN,no,no,no,no,no,1.0,2018,3613.0,PUNJAB,Kapurthala,Kapurthala,15814.0,MANDER DONA,35.0,3.0,206.0,3.448564,21.010001,235.0,120.0,115.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,894.119726,2020.0,B,1,PUNJAB_Kapurthala_Kapurthala_MANDER DONA
1,3630,80,yes,secondary,12,incomplete secondary,secondary,no,no,no,no,no,no,no,no,no,no,no,no,yes,yes,yes,undecided,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1993,1,26,no,sikh,caste,NaN,NaN,no,NaN,no,NaN,no,no,no,no,no,1.0,2019,3630.0,PUNJAB,Jalandhar,Jalandhar - II,14927.0,ATHOLA,34.0,3.0,213.0,6.922049,59.587777,2400.0,1202.0,1198.0,0,0,3.0,3.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,741.609605,2020.0,B,0,PUNJAB_Jalandhar_Jalandhar - II_ATHOLA
2,3630,63,yes,secondary,12,incomplete secondary,NaN,no,yes,no,no,no,no,no,no,no,no,no,no,yes,no,no,declared infecund,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990,7,28,NaN,sikh,caste,NaN,NaN,NaN,NaN,NaN,NaN,no,no,no,no,no,1.0,2019,3630.0,PUNJAB,Jalandhar,Jalandhar - II,14927.0,ATHOLA,34.0,3.0,213.0,6.922049,59.587777,2400.0,1202.0,1198.0,0,0,3.0,3.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,741.609605,2020.0,B,0,PUNJAB_Jalandhar_Jalandhar - II_ATHOLA
3,3630,75,no,secondary,12,incomplete secondary,secondary,no,no,no,no,no,no,no,no,no,no,no,no,yes,yes,yes,no more,2,2.0,no,no,no,no,no,no,no,no,no,no,no,no,1984,6,29,yes,sikh,caste,respondent alone,cash only,yes,for family member,currently working,cash only,no,no,no,no,no,1.0,2019,3630.0,PUNJAB,Jalandhar,Jalandhar - II,14927.0,ATHOLA,34.0,3.0,213.0,6.922049,59.587777,2400.0,1202.0,1198.0,0,0,3.0,3.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,74

In [53]:
print(dhs_ma2['treated'].mean())
# print(dhs_ma_join['Fertility preference'].unique())

0.42947220177319717


In [54]:
dhs_ma_join.to_csv('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/ma2020/dhs1.csv',index=False)